# Memory

Describing memory and getting it synthesized correcly is cumbersome.
Therefore, this will only demonstrate a simple approach to generate memory.
Using [Signal arrays](arrays.ipynb), a simple `r1w1` simplex dual port memory can be inferred to HDL.
This modification enables shared variable output for VHDL to support true dual port behaviour.
The general strategy with the MyIRL synthesis is to use a blackbox element (rely on the library)

In [1]:
from myhdl import intbv
from myirl import *
import random

## Dual port memories

We can not use a `SigArray` for the RAM, because a true dual port RAM would provoke unresolved multiple drivers.
Therefore we hack the `myirl.lists.SigArray` class by deriving and choosing a different element in return of the `__getitem__` member which uses a shared variable.

**Note** Not working in VHDL-2008

In [2]:
from myirl import lists
from myirl.kernel import extensions, utils

from myirl import targets

class CellAssign:  # extensions.ElemAssign
    def __init__(self, parent, portion, val, width):
        self._vref = parent
        self.portion = portion
        self.value = val
        self.width = width
        
    def emit(self, context):
        tgt = context.target
        if tgt.lang != 'VHDL':
            raise TypeError("Unsupported target %s" % type(tgt))
        n, v = self._vref.identifier, self.value
        p = self.portion
        sz = self.width
        context.output("%s(to_integer(%s)) := %s;\n" % (n, p, base.convert(v, tgt, sz)))        

    def get_sources(self, srcs):
        if isinstance(self.value, base.Sig):
            self.value.get_sources(srcs)
    
    def get_drivers(self, drvs):
        pass
    
class ArrayElem(lists.SigIndexed):
    decl_type_vhdl = "shared variable"
    
    def set(self, val):
        w = self.size()
        return CellAssign(self.seq, self.index, val, w)

class RamBuffer(lists.SigArray):    
    def __getitem__(self, item):
        if isinstance(item, (Sig, int)):
            # We can not just return self.val[item], as the iterator
            # has not initialized yet.
            return ArrayElem(self, item)
        else:
            raise TypeError("Multi item slicing of iterator not supported")

Then define a RAM port (legacy class constructs will suffice) and the actual RAM implementation:

In [3]:
class RamPort:
    _inputs = ['we', 'wa', 'ra', 'wd']
    _outputs = ['rd']
    _other = ['clk']
    def __init__(self, AWIDTH, DWIDTH):
        self.clk = ClkSignal()
        self.we = Signal(bool())
        self.ra, self.wa = [ Signal(intbv()[AWIDTH:]) for i in range(2) ]
        self.rd, self.wd = [ Signal(intbv()[DWIDTH:]) for i in range(2) ] 
        
@block
def tdp_ram(pa, pb, INITDATA):
    inst = []
 
    def gen_logic(p, i):
        "Generate port mechanics inline"
        @genprocess(p.clk, EDGE=p.clk.POS)
        def proc():
            yield [
                proc.If(p.we == True).Then(
                    buf[p.wa].set(p.wd)
                ),
                p.rd.set(buf[p.ra])
            ]
        proc.rename("proc%d" % i)
        return proc

    buf = RamBuffer(INITDATA)
    
    for i, p in enumerate([pa, pb]):
        inst.append(gen_logic(p, i))
    
    return inst

### Basic checks

Verify we can evaluate the content:

In [4]:
a = RamBuffer([intbv(i + 0x80)[8:] for i in range(200) ])
b = Signal(intbv(10)[5:])
a[b].evaluate()

intbv(138)

In [5]:
a[0].wire

intbv(128)

In [6]:
from myirl import targets
from myirl.test.common_test import run_ghdl

random.seed(0)

def test():
    RAM_CONTENT = [ intbv(random.randint(0, 2 ** 9))[8:] for i in range(2 ** 9) ]
    pa, pb = [ RamPort(AWIDTH=9, DWIDTH=8) for i in range(2) ]
    inst = tdp_ram(pa, pb, RAM_CONTENT)
    
    f = inst.elab(targets.VHDL)
    run_ghdl(f, inst, std = "93", debug = True) # Note we run with std '93'

In [7]:
test()

 Writing 'tdp_ram' to file /tmp/myirltp9dwr15/tdp_ram.vhdl 
Finished _elab in 0.0017 secs
==== COSIM stdout ====

==== COSIM stderr ====

==== COSIM stdout ====
analyze /home/testing/.local/lib/python3.10/site-packages/myirl-0.0.0-py3.10-linux-x86_64.egg/myirl/targets/../test/vhdl/txt_util.vhdl
analyze /home/testing/.local/lib/python3.10/site-packages/myirl-0.0.0-py3.10-linux-x86_64.egg/myirl/targets/libmyirl.vhdl
analyze /tmp/myirltp9dwr15/tdp_ram.vhdl
elaborate tdp_ram

==== COSIM stderr ====

==== COSIM stdout ====

==== COSIM stderr ====



In [8]:
! grep -A 20 MyIRL {tdp_ram.ctx.path_prefix}/tdp_ram.vhdl 

architecture MyIRL of tdp_ram is
    -- Local type declarations
    -- Signal declarations
    type a_s_3104 is array (0 to 511) of unsigned(7 downto 0);
    shared variable s_3104 : a_s_3104;
begin
    
proc0:
    process(pa_clk)
    begin
        if rising_edge(pa_clk) then
            if (pa_we = '1') then
                s_3104(to_integer(pa_wa)) := pa_wd;
            end if;
            pa_rd <= s_3104(to_integer(pa_ra));
        end if;
    end process;
    
proc1:
    process(pb_clk)
    begin
--
end architecture MyIRL;

